In [51]:
# Generic Libraries
from PIL import Image
from keras.preprocessing.text import Tokenizer
import os
import pandas as pd
import numpy as np
import re,string,unicodedata
import cv2
import requests
import csv
import pickle

#Tesseract Library
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


#Warnings
import warnings
warnings.filterwarnings("ignore")

#Garbage Collection
import gc

#Gensim Library for Text Processing
import gensim.parsing.preprocessing as gsp
from gensim.parsing.preprocessing import remove_stopwords
from gensim import utils

#TextBlob Library (Sentiment Analysis)
from textblob import TextBlob, Word

#Plotting Libraries
import matplotlib.pyplot as plt
import seaborn as sns



In [52]:
#Define Directory Path
#sample_images = r'C:\Users\calli\Downloads\train_images'
test_images = r'C:\Users\calli\Documents\MATLAB\archive\bow_sample'

In [53]:
#Custom Function to Traverse the folder
def traverse(directory):
    path, dirs, files = next(os.walk(directory))
    fol_nm = os.path.split(os.path.dirname(path))[-1]
    print(f'Number of files found in "{fol_nm}" : ',len(files))

In [54]:
#Traversing the folders
#traverse(sample_images)
traverse(test_images)

Number of files found in "archive" :  10


In [55]:
ex_txt = []   #list to store the extracted text
txt4bow = [] #list to use for bag of words

#Function to Extract Text
def TxtExtract(directory):
    """
    This function will handle the core OCR processing of images.
    """
    
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            filepath = subdir + os.sep + file
            
            img_cv = cv2.imread(filepath)
            img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
            
            text = pytesseract.image_to_string(img_rgb)
            
            x = re.sub(r'\n{2, 10}', '\n', text)
            
            ifspace = text.isspace()
            if ifspace == True:
                print(file)
                print("image does not have text")
            else:   
                print(file)
                ex_txt.extend([[file, filepath, (x.rstrip("\n"))]])
                txt4bow.extend([text])
                print(x.rstrip("\n"))
                
    fol_nm = os.path.split(os.path.dirname(subdir))[-1]
    
    print(f"Text Extracted from the files in '{fol_nm}' folder & saved to list..")

In [56]:
#Extracting Text from JPG files in Sample Image Folder
#TxtExtract(sample_images)

#Extracting Text from JPG files in Dataset Folder
TxtExtract(test_images)


37760088-9125741-image-a-4_1610103049128.jpg
Lisa Wilkinson
@Lisa_Wilkinson

The Simpons’ ability to predict the future is now
Officially out of control.

Can | request a nice episode that has Michelle Obama
as President sometime soon please?

 

9:49 AM - Jan 8, 2021 - Twitter for iPad

37767688-0-image-a-8_1610117767171.jpg
 
 
   

y. A

oing to steal|the|Declaration of Independence:

  

When you riotice that all the cops in
Washington DC are distracted by rioters.

486.png
image does not have text
c0y4dmy5by961.jpg
The protesters making their
way inside the Capital
Building after being shot at,
maced, beaten, tear gased,
and tased :

 

images-47-1.jpeg
ey
WITHIN THE BOUNDS OF THE LAW,

>

  

|
‘ir 77

TO KEEP AMER ich SAFE! :

images.jfif
Future: How Do You Want Us To Remember The
Capitol Hil Riots?
Social Studios Text Books:

 

same-cheese-bloodbuzz-me-to-my-son-via-getty-youre-named-after-something-that-was-so-fucking-funny.jfif
 

PY Ryan Liza @
anLizza

To be clear, “v

In [57]:
with open('OCR.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['FileName', 'Filepath', 'Text']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(ex_txt)

In [62]:
#BOW

filtered_txt = []

for n in range(len(txt4bow)):
    #remove stopwords
    filtered_sentence = remove_stopwords(txt4bow[n])
    filtered_txt.extend([filtered_sentence])



# using tokenizer 
model = Tokenizer()
model.fit_on_texts(filtered_txt)

keys = model.word_index.keys()

#print keys 
print(f'keys: {list(keys)}\n')

#create bag of words representation 
rep = model.texts_to_matrix(filtered_txt, mode='count')
print(rep)



keys: ['the', 'to', 'getty', 'lisa', 'wilkinson', 'future', 'y', 'inside', 'building', 'e', 'simpons’', 'ability', 'predict', 'officially', 'control', 'can', 'request', 'nice', 'episode', 'michelle', 'obama', 'president', 'soon', 'please', '9', '49', 'am', 'jan', '8', '2021', 'twitter', 'ipad', 'a', 'oing', 'steal', 'declaration', 'independence', 'when', 'riotice', 'cops', 'washington', 'dc', 'distracted', 'rioters', 'protesters', 'making', 'way', 'capital', 'shot', 'at', 'maced', 'beaten', 'tear', 'gased', 'tased', 'ey', 'within', 'bounds', 'of', 'law', '‘ir', '77', 'keep', 'amer', 'ich', 'safe', 'how', 'do', 'you', 'want', 'us', 'remember', 'capitol', 'hil', 'riots', 'social', 'studios', 'text', 'books', 'py', 'ryan', 'liza', 'anlizza', 'clear', '“via', 'person', 'it', 'means', 'photo', 'comes', 'images', 'charles', 'new', 'year', 'cheese', 'bloodbuzz', 'me', 'son', 'via', "you're", 'named', 'fix', '“ing', 'funny', 'ng', 'january', '6th', 'republicans', 'push', 'false', 'misleading',

In [63]:
with open('BOW.csv', 'w', newline='', encoding='utf-8') as f:
    header = [keys]
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(rep)

In [85]:
#Free up memory
gc.collect()

417